# WOL로 컴퓨터 켜기

* 난이도 : ★★★☆☆☆☆☆☆☆
* 필요라이브러리: struct, socket, os


* Wake On Lan 기능은 랜카드가 대기전력을 사용하여 특정 패킷을 감지하여 패킷이 들어오면 전원을 켤 수 있는 기능입니다.
* 랜카드가 WOL을 지원하여야 하고 바이오스와 윈도우 드라이버 설정에 WOL 기능이 활성화 되어있어야 동작합니다.
* 유선랜에서만 동작합니다.
* 이때 특정 패킷을 매직패킷이라고 부릅니다.
* 매직패킷은 16진수 FF 값이 6번 반복되고 그 뒤에 맥어드레스가 16번 반복된 102 Bytes의 데이터 입니다. 
* 매직패킷은 통상 UDP 7 번, 9번 포트로 전송됩니다.
* 매직패킷의 데이터는 \xFF\xFF... 형식의 C 스타일로 전송되어야 합니다.

## 기본 로직

In [2]:
# \xFF\xFF 형태의 C 스타일로 패킷을 전송하기 위해 사용되는 라이브러리
import struct
# 랜카드로 매직패킷을 보내기 위한 소켓
import socket

# WOL 대상 컴퓨터의 맥어드레스
mac = "AA-BB-CC-DD-EE-FF"

# - 를 기준으로 잘라서 리스트화 시킵니다.
# ['AA', 'BB', 'CC', 'DD', 'EE', 'FF']
address = mac.split("-")

# 문자열 맥어드레스 값을 \xAA\xBB\xCC\xDD\xEE\xFF 형태의
# C 스타일 바이트 형태로 변경해야 합니다.
hw_addr = bytes()

# 리스트의 요소만큼 반복
for addr in address:
    # 16진수 'AA' 값을 10진수로 변경하여 16진수 bytesarray 형태로 포맷팅 합니다.
    # 'AA' -> '\xAA'
    # hw_addr 에 누적시킵니다.
    hw_addr += bytes.fromhex("{:02x}".format(int(addr, 16)))

# 매직패킷 작성
# FF 상수값 6번 + 맥어드레스값 16번 (C 스타일로)
magic = b"\xFF" * 6 + hw_addr * 16
print(magic)

# 매직패킷을 전송할 소켓을 UDP 형태로 생성
# AF_INET 은 InternetProtocol Address Family 로 인터넷 주소체계를 사용하는 소켓이라는 의미...
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# 생성된 소켓에 브로드캐스트 옵션을 설정합니다.
s.setsockopt(socket.SOL_SOCKET, socket.SO_BORADCAST, 1)

# 현재 라우터의 브로드캐스트 아이피의 7번 포트로 매직 패킷을 전송합니다.
# 그러면 라우터는 해당 네트워크 전체에 통보를 하게 되고
# 해당 네트워크에 유선 연결되어 WOL 이 활성화 된 랜카드는 이 통보를 모두 받고
# 매직패킷에 해당하는 맥 어드레스 컴퓨터만 패킷에 반응하여 전원이 켜지게 됩니다.
s.sendto(magic, ('192.168.0.255', 7))

# 소켓은 썼으면 무조건 close()
s.close()

b'\xaa\xbb\xcc\xdd\xee\xff'
b'\xff\xff\xff\xff\xff\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff\xaa\xbb\xcc\xdd\xee\xff'


## 최종 코드

In [ ]:
# \xFF\xFF 형태의 C 스타일로 패킷을 전송하기 위해 사용되는 라이브러리
import struct
# 랜카드로 매직패킷을 보내기 위한 소켓
import socket
import os

# 같은 라우터에 연결된 PC 명과 맥어드레스 목록
mycoms = {
    "pc1": "AA-BB-CC-DD-EE-FF",
    "pc2": "11-22-33-44-55-66",
    "XEON": "2C-FD-A1-34-32-10",
}

def wake_on_lan(mac):
    '''매직 패킷을 전송할 함수

    Args:
        mac (str) : 'AA-BB-CC-DD-EE-FF' 형식의 맥어드레스 값
    '''

    # 맥어드레스를 - 로 스플릿 하여 리스트화 시킵니다.
    addrs = mac.split("-")

    # \xAA\xBB\xCC 형태의 C 스타일 바이트 형태로 변환하기 위해 
    # struct 라이브러리의 pack 함수를 사용합니다.
    # B 는 unsigend char 형태로 파이썬에선 정수형태의 값이 들어갑니다.
    hw_addr = struct.pack("BBBBBB", int(addrs[0], 16), 
                                    int(addrs[1], 16), 
                                    int(addrs[2], 16), 
                                    int(addrs[3], 16), 
                                    int(addrs[4], 16), 
                                    int(addrs[5], 16))

    '''
    # struct 를 안쓰고 이렇게 처리 할 수도 있습니다.

    b1 = bytes.fromhex("{:02x}".format(int(addrs[0], 16)))
    b2 = bytes.fromhex("{:02x}".format(int(addrs[1], 16)))
    b3 = bytes.fromhex("{:02x}".format(int(addrs[2], 16)))
    b4 = bytes.fromhex("{:02x}".format(int(addrs[3], 16)))
    b5 = bytes.fromhex("{:02x}".format(int(addrs[4], 16)))
    b6 = bytes.fromhex("{:02x}".format(int(addrs[5], 16)))
    hw_addr = b1 + b2 + b3 + b4 + b5 + b6

    # for문을 써서 이런식도 가능합니다.
    hw_addr = bytes()
    for addr in addrs:
        hw_addr += bytes.fromhex("{:02x}".format(int(addr, 16)))
    '''

    # FF 상수값 6번 반복 + 맥어드레스값 16번 반복
    # C 스타일이므로 \xFF 형태
    magic = b"\xFF" * 6 + hw_addr * 16

    # SOCK_DGRAM은 UDP 형 소켓 생성
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    # 소켓옵션을 브로드캐스트 형태로 설정
    # SOL_SOCKET 레벨의 옵션 SO_BROADCAST 를 1 로 설정합니다.
    s.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)

    # 라우터(공유기) 255번 아이피(브로드캐스팅) 의 7번 포트로 전송합니다.
    # 라우터에 특별한 설정이 없는한 255번 아이피가 브로드캐스트 아이피가 됩니다.
    # 자세한 사항은 서브넷팅에 관해 검색해 보시기 바랍니다.
    s.sendto(magic, ('192.168.0.255', 7))

    # 소켓 종료
    s.close()

if __name__ == "__main__":
    # 화면을 지웁니다.
    os.system("cls")

    # mycoms.items()는 튜플을 리턴합니다.
    # 따라서 (name, mac) 으로 받아야 처리 됩니다.
    # 인덱스가 필요하므로 enumerate를 사용합니다.
    for i, (name, mac) in enumerate(mycoms.items()):
        print("{}. {} ({})".format(i, mac, name))
    
    # 화면에 출력
    print("=" * 70)

    # 사용자에게 컴퓨터 번호를 입력받습니다.
    select = int(input("매직패킷을 보낼 PC 번호를 입력하세요."))

    # 입력받은 컴퓨터의 맥어드레스 값과 컴퓨터 이름을 구합니다.
    name, mac = list(mycoms.items())[select]

    # 안내 출력
    print("선택된 {} 컴퓨터로 매직패킷을 전송합니다.".format(name))

    # 해당 맥어드레스로 매직패킷을 전송합니다.
    wake_on_lan(mac)
    